
## Урок 6. Задача look-alike
Домашнее задание
1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

import re
import numpy as np
import pandas as pd

### 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [166]:
import re
import numpy as np
import pandas as pd

In [167]:
df = pd.read_csv('adult.data')
df.head(3)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


### 2. сделать feature engineering

Rename the columns 

In [168]:
df = pd.read_csv('adult.data',
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [169]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


No empty rows with this columns 

In [170]:
df.info()  # check for na and isnull the other way
# все колонки заполнены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [171]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


IQR Scores

In [172]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)   # IQR SCORE

age                   20.0
fnlwgt            119224.0
education-num          3.0
capital-gain           0.0
capital-loss           0.0
hours-per-week         5.0
dtype: float64


Number of rows and columns before using the trimmer

In [173]:
df.shape

(32561, 15)

Triming down the outliers within all columns

In [174]:

df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

(18997, 15)

In [175]:
#df.info()

In [176]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,18997.000000,18997.000000,18997.000000,18997.0,18997.0,18997.000000
mean,38.050429,180642.983208,10.246197,0.0,0.0,41.469969
std,12.024558,87002.459482,2.152233,0.0,0.0,3.949900
min,17.000000,14878.000000,5.000000,0.0,0.0,33.000000
25%,28.000000,117496.000000,9.000000,0.0,0.0,40.000000
50%,37.000000,176831.000000,10.000000,0.0,0.0,40.000000
75%,47.000000,228873.000000,12.000000,0.0,0.0,40.000000
max,78.000000,415847.000000,16.000000,0.0,0.0,52.000000


In [177]:
df.sex.dtype

dtype('O')

In [178]:
df.income.dtype

dtype('O')

In [179]:
# replace string values for binary 
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})
#df['sex'] = df['sex'].map({'Male': 0, 'Female': 1})

### Вопрос почему когда я пишу Map для колонки SEX то получаю NAN, поэтому добавил sex в dummies. 

### Почему MAP не работате, проблема с типпом данных ?

In [180]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


Create dummy variables for categorical columns

In [181]:
# df = pd.get_dummies(df.iloc[:, :-1]) the same result
list_of_cols = ['workclass','native-country','education','marital-status','occupation', 'relationship', 'race', 'sex']
df = pd.get_dummies(df, columns= list_of_cols, drop_first=True)

In [182]:
df.head(2)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male
2,38,215646,9,0,0,40,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


### Вопрос не слишком ли много колонок dummies создают для работы с данным.
### Какой еще подход вы посоветовали бы.

### 3. обучить любой классификатор (какой вам нравится)

In [183]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [184]:
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

### Вопрос почему раньше для подсчета ROC AUC мы брали predict_proba а теперь можно только predict

In [185]:

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)
f1 = f1
roc = roc
rec= rec
prc= prc

Classification results:
f1: 88.04%
roc: 82.30%
recall: 87.27%
precision: 88.83%


### А для такого просчёта мы используем predict_proba

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

## 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [187]:
df = pd.read_csv('adult.data',
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [188]:
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

In [189]:
mod_data = pd.get_dummies(df.copy())
mod_data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [190]:
# get the indices of the positives samples
# [0] zero takes out a tuple and gives u an array instead
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)

# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


In [212]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


## ВОПРОС в PU мне пришлось убрать outliers 

df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape
так как я получал ошибку we skip the warning on Categorical/Interval после выше указанной ячейки

В чём была проблема

In [59]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = mod_data.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = mod_data.loc[:, 'income'].values  # original class

### 6. применить random negative sampling для построения классификатора в новых условиях

In [216]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [214]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)


    
f1, roc, rec, prc= evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)
f_2 = f1
roc2 = roc
rec2= rec
prc2= prc

Classification results:
f1: 61.46%
roc: 82.33%
recall: 88.49%
precision: 47.08%


In [219]:
result = {"F-Score":np.array([f1, f1_2]),
          "roc":np.array([roc, roc_2]),
          "Recall":np.array([rec, rec_2]),
        "precision": np.array([prc, prc_2])}

results = pd.DataFrame(result, index=['PN_Classic', 'PU_RandomNS'])
print(results)

                            F-Score       roc    Recall  precision
PN_Classic                 0.614641  0.823305  0.884935   0.470831
PU_RandomNegativeSampling  1.000000  1.000000  1.000000   1.000000
